In [1416]:
import utility

from hmmlearn import hmm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

VERIFY_DAYS = 250
POLICY_DAYS = 30
COMPONENTS = 6

name_list = ['CAT-B', 'ELUX-A', 'ELUX-B', 'ERIC-A', 'ERIC-B', 'HM-B', 
             'ICA', 'KLOV-A', 'KLOV-B', 'TEL2-A', 'TEL2-B', 'TELIA', 'VOLV-A', 'VOLV-B',
             'SAS', 'SAAB-B', 'SEB-A']

#name_list = ['CAT-A', 'CAT-B', 'ELUX-A', 'ELUX-B', 'HM-B', 
#             'ICA', 'KLOV-A', 'KLOV-B', 'TEL2-B', 'TELIA', 'VOLV-A', 'VOLV-B',
#             'SAS', 'SAAB-B', 'SEB-A']

ModuleNotFoundError: No module named 'utility'

In [1403]:
def load_file(file):
    price_list = []
    rate_list = []
    date_list = []
    f=open("./txt-files/" + file, 'r')
    f1 = f.readlines()
    last_price = None
    last_volume = None
    for line in f1:
        info = line.split()
        date = info[0]
        price = int(info[1])
        volume = int(info[2])

        if last_price == None:
            rate = 0
        else:
            rate = price / last_price - 1
        
        if last_volume== None:
            volume_rate = 0
        else:
            volume_rate = volume / last_volume - 1
        
        rate_list.append([rate * 100, volume_rate * 100])
        price_list.append(price)
        date_list.append(date)
        last_price = price
        last_volume = volume
        
    return (price_list, rate_list, date_list)



In [1404]:
import glob, os

data = {}
names = []
succeeded = 0
failed = 0
os.chdir("/Users/xiaodongdai/git_p/aktier")
for root, dirs, files in os.walk("./txt-files"):  
    for filename in files:
        name = filename.replace('.txt', '')
        if name not in name_list:
            continue
        # load the entire sequences from files
        data[name] = {}
        try:
            result = load_file(filename)
            data[name]["price"] = result[0]
            data[name]["rate"] = result[1]
            data[name]["date"] = result[2]
            succeeded += 1
            names.append(name)
        except IndexError:
            print("ignoring file:{} format error".format(filename))
            failed += 1
print("loaded {} files successfully".format(succeeded))
    

loaded 17 files successfully


In [1405]:
# normalization
import statistics as stats

for i in name_list:
    listX = data[i]["rate"]
    price_rate_list = [item[0] for item in listX]
    volume_rate_list = [item[1] for item in listX]

    mean = stats.mean(price_rate_list)
    stddev = stats.stdev(price_rate_list)
    norm_price_rate_list = [(i - mean) / stddev for i in price_rate_list]

    mean = stats.mean(volume_rate_list)
    stddev = stats.stdev(volume_rate_list)
    norm_volume_rate_list = [(i - mean) / stddev for i in volume_rate_list]

    listX = []
    listX = [[norm_price_rate_list[i], norm_volume_rate_list[i]] for i in range(len(norm_price_rate_list))]
    
    print(listX[:10])
    data[i]["rate"] = listX


[[-0.05885723188457897, -0.06338896620254708], [-1.7031329899174987, -0.07646335004542393], [-0.05885723188457897, -0.07367027440174231], [0.7913879250469705, -0.028062345498023163], [-0.4768348930652146, -0.0715929415181485], [1.6272221471152748, 0.03777888616229929], [-0.4665567538558566, -0.06239911999608213], [-0.880995110901036, -0.07457356184390192], [2.0310310740186104, -0.04323948641874639], [-0.8611046783441882, -0.07642919648773924]]
[[-0.016083317727679652, -0.11130888304593342], [-2.7736598806025423, 0.07419831766270124], [-0.7433656674483915, -0.11861753047017445], [-0.782591674061774, -0.11929747872584344], [0.7636001361984037, -0.11988233661602428], [-1.0015940615857988, -0.1115489617714671], [1.3550965751037702, -0.08438187144346114], [-0.5593272640260506, -0.10331754238798432], [-0.016083317727679652, -0.11941827540841193], [0.42377965939633805, -0.10844594424394412]]
[[-0.010487577694736794, -0.18889989386998393], [-3.2818570611626003, 8.166615529158612], [-0.42716520

In [1406]:
# find out the best way 
# TODO: update the model
def check_profit(buy_list, sell_list, price_seq, state_seq, detail=False):
    
    #print("start to check: sell_list:{}, buy_list:{}, price change:{}".format(sell_list, buy_list, 
    #                                                                          price_seq[-1] / price_seq[0] - 1 ))
    money = 10000
    stocks = 0
    for i in range(len(price_seq)):
        if state_seq[i] in buy_list and money > price_seq[i]:
            # we buy in
            stocks_inc = int(money / price_seq[i])
            stocks += stocks_inc
            money = money - stocks_inc * price_seq[i]
            if detail == True:
                print("buy in at day {}: current stocks:{}, current money:{}".format(i,stocks, money))
        elif state_seq[i] in sell_list and stocks > 0:
            # we sell out
            money = money + stocks * price_seq[i]
            stocks = 0
            if detail == True:
                print("sell out at day {}: current stocks:{}, current money:{}".format(i,stocks, money))
    total_asset =  stocks * price_seq[-1] + money
    #print("finished: current stocks:{}, current money:{}, total asset: {}".format(stocks, money, total_asset))
    return total_asset

In [1407]:
def find_optimal_operation(price_list, Y):
    buy_list = []
    sell_list = []
    optimized_buy_list = []
    optimized_sell_list = []
    max_total_asset = 0
    for b in range(COMPONENTS):
        for s in range(COMPONENTS):
            if s != b:
                buy_list.append(b)
                sell_list.append(s)
                total_asset = check_profit(buy_list, sell_list, 
                                           price_list, 
                                           Y)
                if total_asset > max_total_asset:
                    #print("find optimized way:{}".format(max_total_asset))
                    optimized_buy_list.clear()
                    optimized_buy_list.append(b)
                    optimized_sell_list.clear()
                    optimized_sell_list.append(s)
                    max_total_asset = total_asset
                buy_list.clear()
                sell_list.clear()
    
    # 2 buy states, 1 sell  
    for b1 in range(COMPONENTS):
        for b2 in range(COMPONENTS):
            if b2 == b1:
                continue
            for s in range(4):
                if s == b1 or s== b2:
                    continue
                buy_list.append(b1)
                buy_list.append(b2)
                sell_list.append(s)
                total_asset = check_profit(buy_list, sell_list, 
                                           price_list, 
                                           Y)
                if total_asset > max_total_asset:
                    #print("find optimized way:{}".format(max_total_asset))
                    optimized_buy_list.clear()
                    optimized_buy_list.append(b1)
                    optimized_buy_list.append(b2)
                    optimized_sell_list.clear()
                    optimized_sell_list.append(s)
                    max_total_asset = total_asset
                buy_list.clear()
                sell_list.clear()
    # 2 sells , 1 buy
    for b in range(COMPONENTS):
        for s1 in range(COMPONENTS):
            if b == s1:
                continue
            for s2 in range(4):
                if s2 == b or s2 == s1:
                    continue
                buy_list.append(b)
                sell_list.append(s1)
                sell_list.append(s2)
                total_asset = check_profit(buy_list, sell_list, 
                                           price_list, 
                                           Y)
                if total_asset > max_total_asset:
                    #print("find optimized way:{}".format(max_total_asset))
                    optimized_buy_list.clear()
                    optimized_buy_list.append(b)
                    optimized_sell_list.clear()
                    optimized_sell_list.append(s1)
                    optimized_sell_list.append(s2)
                    max_total_asset = total_asset
                buy_list.clear()
                sell_list.clear()
    
    return (optimized_buy_list, optimized_sell_list, max_total_asset/10000)

In [1408]:
# verify_start_date: the find action cannot go over it
# policy_segs how many times we want to check the optimal policies?
def find_multi_optimal_operation(name, price_list, Y, verify_start_date, policy_segs):
    policy_start_date = [0] * policy_segs
    for i in range(policy_segs):
        policy_start_date[i] = verify_start_date - POLICY_DAYS * (i + 1)

    
    data[name]["sell_states_list"] = []
    data[name]["buy_states_list"] = []
    data[name]["maxRate_list"] = []
    
    for i in range(policy_segs):
        end_date = policy_start_date[i] + POLICY_DAYS
        buy_states, sell_states, max_total_asset  = find_optimal_operation(price_list[policy_start_date[i]:end_date], 
                                   Y[policy_start_date[i]:end_date])
        data[name]["sell_states_list"].append(sell_states)
        data[name]["buy_states_list"].append(buy_states)
        data[name]["maxRate_list"].append(max_total_asset)

In [1409]:
# find out the best


# check only num_limit for test
def update_hmm(days_in_future):
    print("重新生成模型")
    for i in range(len(name_list)):
        #print("check no:{}, {}".format(i, name_list[i]))
        name = name_list[i]
        remodel = hmm.GaussianHMM(n_components=COMPONENTS, n_iter=100)
        rate_list = data[name]["rate"]
        price_list = data[name]["price"]
        #print("price_list: {},len:{}".format(price_list, len(price_list)))
        verify_start_date = len(price_list) - days_in_future
        remodel.fit(rate_list[:verify_start_date])

        # predict the first len(inputX) - verify_days days

        Y = remodel.predict(rate_list[:verify_start_date])

        # check the optimized way
        find_multi_optimal_operation(name, price_list, Y, verify_start_date, 2)

        data[name]["hmm"] = remodel
        data[name]["index_learnt"] = verify_start_date
        #print("name: {} index_learnt: {}".format(name, verify_start_date))


In [1410]:
def getSecond(e):
    return e[1]

def get_profit_name_list():
    profit_name_list = []
    for i in range(len(name_list)):
        name = name_list[i]
        maxRate0 = data[name]["maxRate_list"][0]
        maxRate1 = data[name]["maxRate_list"][1]
        if maxRate0 <= 1 or maxRate1 <= 1:
            continue
        if maxRate0 <= 1.2:
            continue
        profit_name_list.append([name, maxRate])
    profit_name_list.sort(key=getSecond, reverse=True)
    return profit_name_list



In [1411]:
def get_stable_name_list():
    stable_name_list = []
    for i in range(num_limit):
        name = names[i]
        rateDiff = abs(data[name]["maxRate_list"][0] - data[name]["maxRate_list"][1])

        stable_name_list.append([name, rateDiff])
    stable_name_list.sort(key=getSecond)
    return stable_name_list


In [1412]:
# find stocks in both lists

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3

def select_stock():
    profit_name_list = get_profit_name_list()
    if len(profit_name_list) == 0:
        print("no stock selected")
        return None
    name = profit_name_list[0][0]
    print("we select a stock:{}, rates: {}, buy: {},{}, sell: {},{}".format(name, 
                                                                      data[name]["maxRate_list"],
                                                                      data[name]["buy_states_list"][0],
                                                                      data[name]["buy_states_list"][1],
                                                                      data[name]["sell_states_list"][0],
                                                                     data[name]["sell_states_list"][1]))
    return name



In [1413]:
def whatToDo(name, days_in_future):
    rate_list = data[name]["rate"]
    price_list = data[name]["price"]
    sell_states = data[name]["sell_states_list"][0]
    buy_states = data[name]["buy_states_list"][0]

    index_learnt = data[name]["index_learnt"]
    #print("index={}".format(index))
    # convert reverse index.

    index = len(rate_list) - days_in_future
    #print("index after={}".format(index))
    sell = False
    buy = False    


    #print("name: {} len_rate_list: {} index:{}, index_learnt:{}".format(name, len(rate_list), index, index_learnt))
    Y = data[name]["hmm"].predict(rate_list[:index])


    #print("Y, buy, sell".format(Y[index-10:index], buy_states, sell_states))
    for db in range(1, 3):
        if Y[index - db] in sell_states:
            sell = True
            break
        elif Y[index - db] in buy_states:
            buy = True
            break

   
    return (sell, buy, price_list[index]/100)
 

In [1414]:
# verify from day: verify_start_date to last day
num_m = 10000
num_s = 0
total_a = num_m
num_buy = 0
num_sell = 0
selected = None
for d in range(VERIFY_DAYS):
    if d % 40 == 0:
        update_hmm(VERIFY_DAYS - d)
        selected = select_stock()
    if num_s != 0:
        sell, buy, price = whatToDo(cur_name, VERIFY_DAYS - d)
        if sell == True:
            num_m = num_m + num_s * price
            num_s = 0
            total_a = num_s * price + num_m
            #print("day: {} s: {} : {}, ast:{}".format(d, name, price, total_a))
            print("day: {} sell: {} : price{}, asset:{}".format(d, cur_name, price, total_a))
            cur_name = None
            num_sell += 1
    if num_s == 0:
        buy == False
        if selected == None:
            continue
        sell, buy, price = whatToDo(selected, VERIFY_DAYS - d)

        #print("{} {} {}".format(sell, buy, price))
        if buy == True:
            num_s_inc = int(num_m / price)
            num_s += num_s_inc
            num_m = num_m - num_s_inc * price
            total_a = num_s * price + num_m
            print("day {} purchase: {} : price{}, asset:{}".format(d, selected, price, total_a))
            num_buy += 1
            cur_name = selected
        
total_a = num_s * price + num_m
print("{} trades，profit: {}".format(num_sell + num_buy, total_a / 10000 - 1))
    

重新生成模型
we select a stock:ERIC-A, rates: [1.4059, 1.094], buy: [4],[5], sell: [1],[0, 1]
day 0 purchase: ERIC-A : price54.3, asset:10000.0
day: 1 sell: ERIC-A : price51.9, asset:9558.400000000001
day 2 purchase: ERIC-A : price51.95, asset:9558.400000000001
day: 5 sell: ERIC-A : price53.25, asset:9796.300000000001
day 6 purchase: ERIC-A : price53.2, asset:9796.300000000001
day: 9 sell: ERIC-A : price55.0, asset:10127.5
day 10 purchase: ERIC-A : price52.9, asset:10127.5
day: 20 sell: ERIC-A : price54.45, asset:10423.550000000001
day 21 purchase: ERIC-A : price53.95, asset:10423.550000000001
day: 28 sell: ERIC-A : price52.5, asset:10143.7
day 29 purchase: ERIC-A : price53.25, asset:10143.7
day: 35 sell: ERIC-A : price55.5, asset:10571.2
day 37 purchase: ERIC-A : price53.55, asset:10571.2
day: 38 sell: ERIC-A : price55.35, asset:10925.800000000003
day 39 purchase: ERIC-A : price54.95, asset:10925.800000000003
重新生成模型
we select a stock:ERIC-A, rates: [1.2247, 1.0846], buy: [1],[0], sell: [2, 